# Imports

In [1]:
import torch
import torch.autograd

from multi_cmd.optim import potentials, cmd_utils

# Linear Price and Cost

**TLDR: linear price function, identical linear cost function, pairwise CGD converges to Nash Equilibrium**

Our profit for each player $i$ is defined as the following:
\begin{gather}
\Pi_i = P\left(\sum_j{q_j}\right) \cdot q_i -C_i(q_i) \\
P(q) = 100 - q \\
C_i(q_i) = 10 \cdot q_i
\end{gather}

Thus, to solve for the Nash equilbrium, we take the first derivative and set it to zero:
\begin{gather}
\frac{\partial\Pi_i}{\partial q_i} = \frac{\partial P\left(\sum_j{q_j}\right)}{\partial q_i} \cdot q_i + P\left(\sum_j{q_j}\right) - \frac{\partial C_i (q_i)}{\partial q_i} = 0
\end{gather}

For the example below, this becomes the following:
\begin{gather}
-1 \cdot q_i + \left(100 - \sum_j {q_j}\right) - 10 = 0
\end{gather}

Solving this analytically, we get $q_i = \frac{45}{2}$ (which is what our algorithm converges to)

In [7]:
# Simple linear price, linear cost game.
def player_payoffs(quantity_list):
    quantity_tensor = torch.stack(quantity_list)
    price = torch.max(100 - torch.sum(quantity_tensor),
                      torch.tensor(0., requires_grad=True))
                      
    payoffs = []
    for i, quantity in enumerate(quantity_tensor):
        # Negative, since CGD minimizes player objectives.
        payoffs.append(- (quantity * price - 10 * quantity))
        
    return payoffs

# Number of iterations and setting up game.
num_iterations = 100
bregman = potentials.shannon_entropy(100)

# Define individual sellers quantities
p1 = torch.tensor([10.], requires_grad=True)
p2 = torch.tensor([20.], requires_grad=True)
p3 = torch.tensor([10.], requires_grad=True)

player_list = [p1, p2, p3]

optim = cmd_utils.CMD([[p1], [p2], [p3]], bregman=bregman)

for i in range(num_iterations):
    payoffs = player_payoffs(player_list)
    optim.step(payoffs)

print(player_list)
print(payoffs)


[tensor([22.5000], requires_grad=True), tensor([22.5001], requires_grad=True), tensor([22.5000], requires_grad=True)]
[tensor([-506.2484], grad_fn=<NegBackward>), tensor([-506.2491], grad_fn=<NegBackward>), tensor([-506.2484], grad_fn=<NegBackward>)]


## Quadratic Price Function

**TLDR: quadratic price function, identical linear cost function, pairwise CGD converges to Nash Equilibrium (with learning rate tuning)**

Our profit for each player $i$ is defined as the following:
\begin{gather}
\Pi_i = P\left(\sum_j{q_j}\right) \cdot q_i -C_i(q_i) \\
P(q) = 100 - \sum_j{q_j^2} \\
C_i(q_i) = 10 \cdot q_i
\end{gather}

Thus, to solve for the Nash equilbrium, we take the first derivative and set it to zero:
\begin{gather}
\frac{\partial\Pi_i}{\partial q_i} = \frac{\partial P\left(\sum_j{q_j}\right)}{\partial q_i} \cdot q_i + P\left(\sum_j{q_j}\right) - \frac{\partial C_i (q_i)}{\partial q_i} = 0
\end{gather}

For the example below, this becomes the following:
\begin{gather}
-2 \cdot q_i^2 + \left(100 - \sum_j {q_j^2}\right) - 10 = 0 \\
90 = \sum_{i\neq j} {q_j^2} - 3 q_i^2
\end{gather}

Solving this, we have multiple Nash Equlibrium, but the only solution with all non-negative quantities (as defined by our bregman divergence constraints), we get $q_i = \sqrt{18} = 4.24$ (which is what our algorithm converges to).

In [8]:
# Simple quadratic price, linear cost game.
def player_payoffs(quantity_list):
    quantity_tensor = torch.stack(quantity_list)
    price = torch.max(
        100 - torch.sum(torch.pow(quantity_tensor, 2)),
        torch.tensor(0., requires_grad=True)
    )
                      
    payoffs = []
    for i, quantity in enumerate(quantity_tensor):
        # Negative, since CGD minimizes player objectives.
        payoffs.append(- (quantity * price - 10 * quantity))
        
    return payoffs

# Number of iterations and setting up game.
num_iterations = 100
bregman = potentials.shannon_entropy(100)

# Define individual sellers quantities
p1 = torch.tensor([1.], requires_grad=True)
p2 = torch.tensor([2.], requires_grad=True)
p3 = torch.tensor([1.], requires_grad=True)

player_list = [p1, p2, p3]

optim = cmd_utils.CMD([[p1], [p2], [p3]], bregman=bregman)

for i in range(num_iterations):
    payoffs = player_payoffs(player_list)
    optim.step(payoffs)

print(player_list)
print(payoffs)


[tensor([4.2426], requires_grad=True), tensor([4.2426], requires_grad=True), tensor([4.2426], requires_grad=True)]
[tensor([-152.7349], grad_fn=<NegBackward>), tensor([-152.7354], grad_fn=<NegBackward>), tensor([-152.7349], grad_fn=<NegBackward>)]


## Non-linear Cost Function

**TLDR: linear price function, identical non-linear cost function (simulating at-scale production), pairwise CGD converges to Nash Equilibrium**

Our profit for each player $i$ is defined as the following:
\begin{gather}
\Pi_i = P\left(\sum_j{q_j}\right) \cdot q_i -C_i(q_i) \\
P(q) = 100 - q \\
C_i(q_i) = 10 \cdot \left(\frac{10}{x+10}\right)
\end{gather}

Thus, to solve for the Nash equilbrium, we take the first derivative and set it to zero:
\begin{gather}
\frac{\partial\Pi_i}{\partial q_i} = \frac{\partial P\left(\sum_j{q_j}\right)}{\partial q_i} \cdot q_i + P\left(\sum_j{q_j}\right) - \frac{\partial C_i (q_i)}{\partial q_i} = 0
\end{gather}

For the example below, this becomes the following:
\begin{gather}
-1 \cdot q_i + \left(100 - \sum_j {q_j}\right) - \frac{1000}{(q_i+10)^2} = 0
\end{gather}

Solving this analytical equation (using MATLAB/Octave), we have multiple Nash Equilibrium, but the only solution with all non-negative quantities, we get $q_i = 24.793$ (which is what our algorithm converges to). 

```
syms a positive
syms b positive
syms c positive

[sol_a, sol_b, sol_c] = solve(...
    100 - 2*a - b - c - 1000/((a+10)^2), ...
    100 - a - 2*b - c - 1000/((b+10)^2), ...
    100 - a - b - 2*c - 1000/((c+10)^2) ...
)
```


In [10]:
# Simple quadratic price, linear cost game.
def player_payoffs(quantity_list):
    quantity_tensor = torch.stack(quantity_list)
    price = torch.max(
        100 - torch.sum(quantity_tensor),
        torch.tensor(0., requires_grad=True)
    )
                      
    payoffs = []
    for i, quantity in enumerate(quantity_tensor):
        # Negative, since CGD minimizes player objectives.
        payoffs.append(- (quantity * price - 100 * quantity / (quantity + 10)))
        
    return payoffs

# Number of iterations and setting up game.
num_iterations = 100
bregman = potentials.shannon_entropy(100)

# Define individual sellers quantities
p1 = torch.tensor([1.], requires_grad=True)
p2 = torch.tensor([2.], requires_grad=True)
p3 = torch.tensor([1.], requires_grad=True)

player_list = [p1, p2, p3]

optim = cmd_utils.CMD([[p1], [p2], [p3]], bregman=bregman)

for i in range(num_iterations):
    payoffs = player_payoffs(player_list)
    optim.step(payoffs)

print(player_list)
print(payoffs)


[tensor([24.7935], requires_grad=True), tensor([24.7936], requires_grad=True), tensor([24.7935], requires_grad=True)]
[tensor([-563.9376], grad_fn=<NegBackward>), tensor([-563.9387], grad_fn=<NegBackward>), tensor([-563.9376], grad_fn=<NegBackward>)]
